In [1]:
# Load LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# load ascii text and covert to lowercase
filename = "teste.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [3]:
palavras_texto = raw_text.split()

In [4]:
print(palavras_texto[:10])

['contos', 'para', 'a', 'infancia', 'escolhidos', 'dos', 'melhores', 'auctores', 'por', 'guerra']


In [5]:
len(palavras_texto)

27123

In [6]:
palavras = sorted(list(set(palavras_texto)))

In [7]:
print(palavras[:10])

['(_levanta-se_.)', '(olhando', '(olhando).--\xc3\x89', '*', '*a', '*alberto*', '*boa', '*branca', '*carlos', '*como']


In [8]:
palavra_to_int = dict((c,i) for i, c in enumerate(palavras))

In [9]:
int_to_palavra = dict((i, c) for i, c in enumerate(palavras))

In [10]:
n_palavras = len(palavras_texto)

In [11]:
n_vocab = len(palavras)

In [12]:
print("Total Characters: ", n_palavras)
print("Total Vocab: ", n_vocab)

('Total Characters: ', 27123)
('Total Vocab: ', 7793)


In [13]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []

In [14]:
for i in range(0, n_palavras - seq_length, 1):
    seq_in = palavras_texto[i:i + seq_length]
    seq_out = palavras_texto[i + seq_length]
    dataX.append([palavra_to_int[palavra] for palavra in seq_in])
    dataY.append(palavra_to_int[seq_out])
n_patterns = len(dataX)    

In [15]:
print("Total Patterns: ", n_patterns)

('Total Patterns: ', 27023)


In [16]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

In [17]:
X.shape

(27023, 100, 1)

In [18]:
# normalize
X = X / float(n_vocab)

In [19]:
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [20]:
# define the LSTM model
model = Sequential()

In [21]:
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))

In [22]:
model.add(Dropout(0.2))

In [23]:
model.add(Dense(y.shape[1], activation='softmax'))

In [25]:
# load the network weights
filename = "palavras-713-1.8762.hdf5"
model.load_weights(filename)

In [26]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [48]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ' '.join([int_to_palavra[value] for value in pattern]), "\""
print "\""
# generate characters
for i in range(50):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_palavra[index]
	seq_in = [int_to_palavra[value] for value in pattern]
	sys.stdout.write(result + " ")
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print "\nDone."

Seed:
" á noite a parede da quinta. uma vez deu prova d'uma extraordinaria sagacidade; um jornaleiro, que se empregava muitas vezes em levar saccos de trigo da quinta para casa, tentou de noite roubar um sacco. piloto, que o conhecia, não fez a menor demonstração de hostilidade emquanto o homem seguiu o caminho da quinta, mas, desde que se afastou tomando por outra estrada, o guarda vigilante agarrou-o pela blusa sem o largar. era como se dissesse: «onde vaes tu com o trigo de meu dono?» o ladrão quiz pôr então outra vez o sacco d'onde o tinha tirado; piloto não "
"
consentiu, que teve-o e homem, o a a e aproveitàra que que a e o perdeu dez o um a mas admiração que que a que mudára que e que que que a a que a fria. espere!... o agulha e o e que o de de que que de 
Done.
